# Lab 7: The Bootstrap

## Introduction

In homework this week, you will examine the bootstrap in greater detail. This lab gives a jump start on the homework.  The goal is to develop a functional approach to bootstrapping any statistic for any sample.


In [1]:
# Run this cell to set up your notebook

import seaborn as sns
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
sns.set()

from IPython.display import display, Latex, Markdown
from client.api.notebook import Notebook
ok = Notebook('lab07.ok')


Assignment: lab07
OK, version v1.12.10



In [2]:
ok.auth(force=True) # Change False to True if you are getting errors authenticating


Open the following URL:

https://okpy.org/client/login/

After logging in, copy the code from the web page and paste it into the box.
Then press the "Enter" key on your keyboard.

Paste your code here: 07VdONVahn2wYk3EaMiCEC3hBzSOW9
Successfully logged in as yining.jiang@berkeley.edu


## The Bootstrap Procedure

The plain-vanilla bootstrap is a very simple process: 
* Sample with replacement from the original sample (now the bootstrap population). These samples are called *bootstrap samples*. We typically take thousands of bootstrap samples (~10,000 is common).
* Calculate the statistic of interest for each bootstrap sample. This statistic is called the *bootstrap statistic*, and the empirical distribution of these bootstrap statistics is an approximation to the *sampling distribution* of the bootstrapped statistic.

Your task is to write a generic boostrap procedure. 

### Question 1


First, write your own sampling function. The function `simple_resample` samples with replacement from the integers 0 through *n-1* and returns an array of length *n*
with the sampled integers. That is, `simple_resample` produces the indices for
a single bootstrap replicate from the bootstrap population.

Use the *numpy.random* function *randint* to do the random sampling. 


In [5]:
# given:  n, an integer
# return: an array of length n of a random sample with replacement of
#         the integers 0, 1, ..., n-1 
def simple_resample(n):
    return(np.random.randint(0,n-1,n))

Try your `simple_resample` function on a few cases to see if the sampling looks right.

In [6]:
simple_resample(10)

array([0, 6, 5, 7, 0, 4, 6, 5, 3, 4])

In [7]:
_ = ok.grade('q01')
_ = ok.backup()

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 1
    Failed: 0
[ooooooooook] 100.0% passed



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saving notebook... Saved 'lab07.ipynb'.
Backup... 100% complete
Backup successful for user: yining.jiang@berkeley.edu
URL: https://okpy.org/cal/ds100/fa17/lab07/backups/qxy0xk
NOTE: this is only a backup. To submit your assignment, use:
	python3 ok --submit



### Question 2

Next write the function `bootstrap` which returns an array of length `replicates`, each entry being the `statistic` of interest computed on a bootstrap `resample` of the `boot_pop` (bootstrap population).

In [23]:
a=np.array([[1,1,1],[2,2,2]])

In [24]:
# given:  boot_pop, an array of shape n x d,
#         statistic, a function which takes boot_pop and returns a number,
#         resample, a function which takes n and returns a random sample from the integers [0, n)
#         replicates, the number of resamples 
# return: an array of length replicates, each
#          entry being the statistic computed
#          on a bootstrap sample of the data.
def bootstrap(boot_pop, statistic, resample, replicates = 10000):
    n=boot_pop.size
    samples=[]
    for i in range(replicates):
        samples.append(statistic(boot_pop[resample(n)]))
    return np.array(samples)


In [25]:
_ = ok.grade('q02')
_ = ok.backup()

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
q02 > Suite 1 > Case 1

>>> (len(bootstrap(np.ones(10), np.mean, simple_resample)) == 10000)
True
>>> (np.all(bootstrap(np.ones(10), np.mean, simple_resample, replicates = 1000) == np.ones(1000)))
True
>>> # chances are less than 3.9 in 10^22 that this test fails when the function is correct
>>> (sum(bootstrap(np.array([0,1]), np.median, simple_resample, replicates = 10000) == 0.5) in range(5000-500,5000+500))
False

# Error: expected
#     True
# but got
#     False

Run only this test case with "python3 ok -q q02 --suite 1 --case 1"
---------------------------------------------------------------------
Test summary
    Passed: 0
    Failed: 1
[k..........] 0.0% passed



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saving notebook... Saved 'lab07.ipynb'.
Backup... 100% complete
Backup successful for user: yining.jiang@berkeley.edu
URL: https://okpy.org/cal/ds100/fa17/lab07/backups/rkzxRL
NOTE: this is only a backup. To submit your assignment, use:
	python3 ok --submit



### Question 3

Below is a sample of body temperatures for 50 male university students.

Use your `bootstrap` and `simple_resample` functions to find a 95% bootstrap confidence interval for the population mean, i.e., the average body temperature of male university students.  That is construct a length  2-tuple called `conf_int` that contains the 2.5th and 97.5th percentile of the bootstrap sampling distribution of the sample average.


In [27]:
body_temp = np.array([96.3, 96.7, 96.9, 97.0, 97.1, 97.1, 97.1, 97.2, 97.3, 97.4, 97.4, 97.4,
                 97.4, 97.5, 97.5, 97.6, 97.6, 97.6, 97.7, 97.8, 97.8, 97.8, 97.8, 97.9,
                 97.9, 98.0, 98.0, 98.0, 98.0, 98.1, 98.1, 98.2, 98.2, 98.6, 98.6, 98.6,
                 98.6, 98.6, 98.6, 98.7, 98.7, 98.8, 98.8, 98.8, 98.9, 99.0, 99.0, 99.0,
                 99.1, 99.2])

In [28]:
bootstrap_stats = bootstrap(body_temp,np.mean,simple_resample, replicates = 10000)

conf_int = np.array([np.percentile(bootstrap_stats,2.5),np.percentile(bootstrap_stats,97.5)])

## Confidence Intervals based on normal theory

The central limit theorem says that the sample mean has a distribution that roughly follows the normal distribution.  The central limit theorem holds when the sample size is large in absolute terms, but small relative to the population. 

### Question 4

If we assume that the sampling distribution of the sample average is normally distributed, then we can use this property to create a confidence interval. The sampling 
distribution of a statistic is often approximately normally distributed. 

For a standard normal distribution, which is centered at 0 and has a spread of 1, we can find the percentiles with the `ppf` method of  `scipy.stats.norm`.   


Use `ppf` to find the 0.975 and 0.025 quantiles of the standard normal distribution.

In [29]:
from scipy.stats import norm

q975 = norm.ppf(0.975)
q025 = norm.ppf(0.025)

By symmetry, we know that the 2.5 percentile is the negative of the 97.5 percentile, ie., they are $-1.96$ and $1.96$.  This means that the area under the standard normal curve between $-1.96$ and $1.96$ is 95%. We can use this property to form a confidence interval for the population mean.

If $\theta$ is the
population mean, and we estimate it with our sample mean, $\hat{\theta}$, then for a simple 
random sample we know that

$$
E(\hat{\theta}) = \theta
$$
and 
$$
SE(\hat{\theta}) = \sigma / \sqrt{n},
$$
where $\sigma$ is the population SD.

We also have that $\hat{\theta}$ has a normal distribution 
(we're assuming $n$ is large enough).
This means that we can standardize $\hat{\theta}$ as follows
$$ \frac{\hat{\theta} - \theta} {SE({\hat{\theta}})}$$

Now, the standardized statistics has a standard normal distribution, so 
$$0.95 = {\cal P}\left(-1.96 \leq \frac{\hat{\theta} - \theta} {SE({\hat{\theta}})} \leq 1.96\right)$$

We can reararnge terms to find a confidence interval for $\theta$
\begin{eqnarray*}
0.95 &=& {\cal P}\left(-1.96 \leq \frac{\hat{\theta} - \theta} {SE({\hat{\theta}})} \leq 1.96\right) \\
 & = & {\cal P}\left(-1.96 SE({\hat{\theta}}) \leq {\hat{\theta} - \theta} \leq 1.96 SE({\hat{\theta}})\right) \\
 & = & {\cal P}\left(\hat{\theta} - 1.96SE({\hat{\theta}}) \leq {\theta} \leq \hat{\theta} + 1.96SE({\hat{\theta}})\right) 
\end{eqnarray*}

Thus the interval $(\hat{\theta} - 1.96SE({\hat{\theta}}),
\hat{\theta} + 1.96SE({\hat{\theta}}))$ is a 95%
confidence interval for $\theta$.

Often we don't know the $\sigma$, and so we can't plug $\sigma/\sqrt{n}$ in for $SE(\hat{\theta})$ in the above interval.  Instead we use the sample SD as an estimate for $\sigma$. When we do this, the distribution of 

$$ \frac{\hat{\theta} - \theta} {SD(\texttt{sample})/\sqrt{n}}$$

is no longer normal. It now follows a $t$-distribution. The $t$-distribution is actually a family of distributions. it is parameterized by the "degrees of freedom", which in our case is $n-1$.  The $t$-distribution is symmetric and unimodal and looks much like the normal distribution, except the tails are somewhat fatter than the normal tails. As $n$ increases, the sample SD approaches the population SD, $\sigma$ and the tails of the $t$-distribution get less fat and converge to the normal tails.

With the $t$-distribution, the 95% confidence interval becomes:

$$(\hat{\theta} + t_{0.025, n-1} \frac{SD(\texttt{sample})}{\sqrt{n}}, ~~~\hat{\theta} + t_{0.975, n-1} \frac {SD(\texttt{sample})}{\sqrt{n}})$$



### Question 5

Find the 0.975 quantile of the $t$-distribution for various sample `sizes`. How close are they to the quantiles of the normal?

In [30]:
from scipy.stats import t

sizes = np.array([6, 12,  24, 32, 100, 200])

#Hint: use the ppf function again
left_endpoint =-t.ppf(0.025,sizes)

In [31]:
_ = ok.grade('q05')
_ = ok.backup()

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 1
    Failed: 0
[ooooooooook] 100.0% passed



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saving notebook... Saved 'lab07.ipynb'.
Backup... 100% complete
Backup successful for user: yining.jiang@berkeley.edu
URL: https://okpy.org/cal/ds100/fa17/lab07/backups/pYxvkV
NOTE: this is only a backup. To submit your assignment, use:
	python3 ok --submit



### Question 6

Use the $t$-distribution to create a 95% confidence interval for the population mean for the data in Question 3. Compare the lengths of these two confidence intervals.

To do this, follow the steps

* Find the mean and SD of the sample
* Find the percentiles for the $t$ with 49 degrees of freedom
* Construct the confidence interval using the formula above


In [32]:
temp_mean = np.mean(body_temp)
temp_sd = np.std(body_temp)

t_percentiles = lambda x:t.ppf(x,49)

#The confidence interval should be represented as an array containing the left end point and the right end point.
conf_int_t = np.array([temp_mean+t_percentiles(0.025)*temp_sd/np.sqrt(50),temp_mean+t_percentiles(0.975)*temp_sd/np.sqrt(50)])

In [33]:
_ = ok.grade('q06')
_ = ok.backup()

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Running tests

---------------------------------------------------------------------
Test summary
    Passed: 1
    Failed: 0
[ooooooooook] 100.0% passed



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saving notebook... Saved 'lab07.ipynb'.
Backup... 100% complete
Backup successful for user: yining.jiang@berkeley.edu
URL: https://okpy.org/cal/ds100/fa17/lab07/backups/82RvE3
NOTE: this is only a backup. To submit your assignment, use:
	python3 ok --submit



Compare `conf_int` based on the bootstrap percentile method and `conf_int_t` based on the $t$-distribution.  

Do they have the same length? The same endpoints?

## Submission

Run the cell below to run all the OkPy tests at once:

In [ ]:
import os
print("Running all tests...")
_ = [ok.grade(q[:-3]) for q in os.listdir("ok_tests") if q.startswith('q')]

Now, run the cell below to submit your assignment to OkPy. The autograder should email you shortly with your autograded score. The autograder will only run once every 30 minutes.

**If you're failing tests on the autograder but pass them locally**, you should simulate the autograder by doing the following:

1. In the top menu, click Kernel -> Restart and Run all.
2. Run the cell above to run each OkPy test.

**You must make sure that you pass all the tests when running steps 1 and 2 in order.** If you are still failing autograder tests, you should double check your results.

In [ ]:
_ = ok.submit()